In [1]:
import os,sys,re
import numpy as np
import pandas as pd

In [4]:
def create_ret_dict(PriceVolume_dr,universe,close_col):
    #close_col = "Adjusted_close"
    tickers_pv = {t: pd.read_csv(os.path.join(PriceVolume_dr, t + ".csv")) for t in universe}
    bad_tickers = []
    for k in tickers_pv.keys():
        try:
            if  tickers_pv[k][close_col].isnull().values.any():
                bad_tickers.append(k)
            tickers_pv[k][close_col] = tickers_pv[k][close_col].fillna(0.00)
            tickers_pv[k]["return"] = getattr(tickers_pv[k][close_col], "pct_change")()
            tickers_pv[k] = tickers_pv[k].set_index("Date")
        except Exception as e:
            print(e, " ", k)
            print("in create_ret_dict")
            bad_tickers.append(k)
    print("bad ", bad_tickers)
    return tickers_pv

In [7]:
def between_quarters(filename,index_data_path):
    df1 = pd.read_csv(filename)
    r1 = os.path.basename(filename).split(".")[0]
    dt1 = re.findall('20[0-9]+',r1)[0]
    start_date = dt1[:4]+"-"+dt1[4:6]+"-"+dt1[6:8]
    universe = list(df1["Ticker"])[:-2]
    universe = [x for x in universe if x not in [ "HFC","BLKFDS"]]
    close_col = "Adjusted_close"#"Close"
    PriceVolume_dr = os.path.join("..","data","PriceVolume")
    ret_d = create_ret_dict(PriceVolume_dr,universe,close_col)
    d1 = df1.set_index("Ticker")["Weight (%)"].to_dict()
    d11 = {x:d1[x] for x in d1.keys() if x not in ['WCG','ESH0','ESZ9','ESU9','XTSLA','ESM9','ESH9','ESZ8','USD','HFC','SIG','PDCO', 'UAA', 'UA', 'NWS','ESU2','ESU8',"ESH8","UBFUT","BLKFDS"]}
    nrmlz = 1./sum(d11.values())
    d11 = {x:d11[x]*nrmlz for x in d11.keys()}
    start_d = start_date
    if int(start_d.split("-")[1]) < 12:
        if int(start_d.split("-")[1]) < 9:
            end_d = start_d[:4]+"-"+"0"+str(int(start_d.split("-")[1])+3)+"-"+start_d[-2:]
        else:
            end_d = start_d[:4]+"-12-"+start_d[-2:]
    else:
        end_d = str(int(start_d.split("-")[0])+1) + "-03-"+start_d[-2:]
        
    #end_d = "2021-06-22"
    target_index = pd.read_csv(index_data_path)
    d12 = {x:d11[x] for x in d11.keys() if x in ret_d.keys() and ret_d[x][start_d:].shape[0]>0}
    nrmlz2 = 1./(sum(d12.values()))
    print(sum(d12.values()))
    d12 = {x:nrmlz2*d12[x] for x in d12.keys()}
    print(nrmlz2)
    d12 = {x:d11[x] for x in d11.keys() if x in ret_d.keys() and ret_d[x][start_d:].shape[0]>0}
    nrmlz2 = 1./(sum(d12.values()))
    print(sum(d12.values()))
    d12 = {x:nrmlz2*d12[x] for x in d12.keys()}
    print(nrmlz2)
    A11 = {x: d12[x] * (ret_d[x]["return"][start_d:end_d].fillna(0.0)+1.).cumprod() for x in list(d12.keys())[:]  }
    index_df =  pd.read_csv(index_data_path)
    index_df = index_df.set_index("Date")
    index_df["return"] = index_df["Adjusted_close"].pct_change().fillna(0.0)
    return pd.DataFrame(A11).fillna(0.0).sum(axis=1),(index_df["return"][start_d:end_d]+1.).cumprod()


In [8]:
%%time
filename = os.path.join("..","data","holdings","IVV","IVV_holdings_20190622_f.csv")
index_data_path = os.path.join("..","data","index_data","IVV.csv")
dfx1,dfx2 = between_quarters(filename,index_data_path)

'Adjusted_close'   XTSLA
in create_ret_dict
bad  ['XTSLA']
0.9947853991175294
1.0052419354838706
0.9947853991175294
1.0052419354838706
CPU times: user 10.2 s, sys: 2.1 s, total: 12.3 s
Wall time: 16.7 s


In [9]:
dfx1.tail()

Date
2019-09-16    1.017597
2019-09-17    1.020182
2019-09-18    1.020557
2019-09-19    1.020541
2019-09-20    1.015616
dtype: float64

In [10]:
dfx2.tail()

Date
2019-09-16    1.021687
2019-09-17    1.024157
2019-09-18    1.024901
2019-09-19    1.024766
2019-09-20    1.019116
Name: return, dtype: float64